In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 626.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 56.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.5.0.post0 requires torch>=2.1.0, but you have torch 1.13.1+cu117 which is incompatible.
torchmetrics 1.6.1 requires torch>=2.0.0, but you have torch 1.13.1+cu117 which

In [2]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 131 (delta 0), reused 0 (delta 0), pack-reused 129 (from 2)
Receiving objects: 100% (131/131), 1.13 MiB | 8.21 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
%cd stylegan2-ada-pytorch

/kaggle/working/stylegan2-ada-pytorch


In [4]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import json
from torchvision import transforms

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [5]:
def prepare_fer2013_with_json(csv_file, output_dir):
    df = pd.read_csv(csv_file)
    df = df[df['Usage'] == 'Training']
    os.makedirs(output_dir, exist_ok=True)
    resize_transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((64, 64)), transforms.ToTensor()])
    image_data = {}
    for i, row in df.iterrows():
        label = row['emotion']
        pixels = row['pixels']
        img = np.array([int(p) for p in pixels.split()], dtype=np.uint8).reshape(48, 48)
        img = resize_transform(img)
        img = (img * 255).numpy().astype(np.uint8).squeeze(0)
        img_filename = f"{i:05d}.png"
        img_path = os.path.join(output_dir, img_filename)
        Image.fromarray(img).save(img_path)
        image_data[img_filename] = int(label)
    dataset_json = {'labels': [[fname, [label]] for fname, label in image_data.items()]}
    with open(os.path.join(output_dir, 'dataset.json'), 'w') as f:
        json.dump(dataset_json, f)
    print(f"Prepared data (64x64) in {output_dir}")

prepare_fer2013_with_json("/kaggle/input/fer2013comp/fer2013.csv", "data/fer2013_images_64")

Prepared data (64x64) in data/fer2013_images_64


In [6]:
!python train.py \
  --outdir=/kaggle/working/training-runs \
  --data=data/fer2013_images_64 \
  --gpus=2 \
  --cfg=auto \
  --gamma=1 \
  --mirror=1 \
  --metrics=fid50k_full,is50k,pr50k3_full \
  --kimg=10000 \
  --workers=4 \
  --cond=1 \
  --snap=50 \
  --aug=fixed \
  --p=0.5 \
  > training_log10.txt 2>&1

In [7]:
# !python train.py \
#   --outdir=/kaggle/working/training-runs \
#   --data=data/fer2013_images_64 \
#   --gpus=2 \
#   --cfg=auto \
#   --gamma=1 \
#   --mirror=1 \
#   --metrics=fid50k_full,is50k,pr50k3_full \
#   --kimg=10000 \
#   --workers=4 \
#   --cond=1 \
#   --snap=50 \
#   --aug=noaug \
#   --resume=/kaggle/input/stylegan2016x64/pytorch/default/1/network-snapshot-002016.pkl \
#   > training_log_64_optimized.txt 2>&1